In [1]:
# DONE: Hello World
# DONE: Crawl over stored Files
# DONE: Operate Elastic Search Class
# DONE: Populate wiki_evidence
# DONE: How to manage a peristant volume
# TODOs: Information Retrieval program, given search and rank sentences.
# TODOs: Implement DELETE on Database Class
# https://stackoverflow.com/questions/34621093/persist-elastic-search-data-in-docker-container
# TODOs: Repurpose DrQA Elasticsearch Module
# TODOs: Run Full-Evidence
# TODOs: Nice Logging

In [5]:
### CRAWL FILE-SYSTEM ###
import os
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

knowledge_path = "/Users/joshua.sheppard/wiki_extract_II"

def iter_filesys(path):
    if os.path.isfile(path):
        yield path

    elif os.path.isdir(path):
        for dir, _, filenames in os.walk(path):
                for f in filenames:
                    if not f.endswith('.DS_Store'):
                        yield os.path.join(dir, f)

    else:
        raise RuntimeError("Invalid path %s" % path)

kw_files = iter_filesys(knowledge_path)

kw_sample = []
for i in kw_files:
    kw_sample.append(i)

# print(kw_sample[:-3])
print(len(kw_sample))

17039


In [4]:
### UPLOAD USING GENERATOR ###
from multiprocessing import Pool
from utils import get_contents
import utils
from tqdm import tqdm
import json

# TODOs: Apply Paralell Multiprocess
# def get_contents_(filename):
#     """Parse the contents of a file. Each line is a JSON encoded document."""
#     documents = []

#     with open(filename) as f:
#         for line in f:
#             doc = json.loads(line)

#             if doc["text"] == "": continue
#             if not doc: continue

#             documents.append((doc['id'], doc["title"], doc["text"]))

#     return documents

# files = [f for f in kw_sample]

# def generate_data():
#     with tqdm(total=len(files)) as pbar:
#         #for document in p.map(utils.get_contents_2, files):
#         for file in files:
#                 #yield get_contents_2(file)
#                 docs = get_contents_(file)
#                 for doc in docs:
#                     yield(doc)

#         pbar.update()

# generate_data()
# wiki_data = [i for i in generate_data()]

In [5]:
# len(wiki_data) 

In [5]:
# knowledge = wiki_data[0]
# knowledge

In [6]:
import json
from multiprocessing.pool import ThreadPool as Pool
from tqdm import tqdm
import spacy

files = [f for f in kw_sample]
nlp = spacy.load("en_core_web_sm")

def get_contents_(filename):
    """Parse the contents of a file. Each line is a JSON encoded document."""
    documents = []

    with open(filename) as f:
        for line in f:
            doc = json.loads(line)

            if doc["text"] == "": continue
            if not doc: continue

            documents.append((doc['id'], doc["title"], doc["text"]))

    return documents

def generate_data():
    #with tqdm(total=len(files)) as pbar:
        #for document in p.map(utils.get_contents_2, files):
        for file in files:
                #yield get_contents_2(file)
                docs = get_contents_(file)
                for doc in docs:
                    yield(doc)

        #pbar.update()

import more_itertools
def sentence_window(article, window=5, step=2): 
    """ Generates a list of sentences of sliding size = window """
    
    sents = list(nlp(article).sents)
    
    if len(sents) == window:
        yield str(sents)

    for window in more_itertools.windowed(sents, n=window, step=2):
        yield window

    # for i in range(len(sents) - window + 1):
    #     #print(sents[i:i + window])
    #     #yield(sents[i:i + window])
    #     yield(sents[i:i + window])

# generate_data()
# wiki_data_passages = []
# count = 0

pool = Pool(8)

# def to_raw(string):
#     return fr"{string}"

import re
def clean(passage):
    passage = str(passage)
    #passage = to_raw(passage)
    passage.encode("unicode_escape")
    passage.replace('"', '"')
    passage.strip()
    passage = re.sub("\n", "", passage)
    passage = re.sub('"', "'", passage)
    
    return passage    
    
def passages(idx, source):
    count = 0
    with tqdm(total=len(files)) as pbar:
        for i in pool.apply(generate_data):
            count += 1
            id, title, article = i

            for window in sentence_window(article):
                passage = " ".join(clean(passage) for passage in window)

                if len(passage) < 50: continue
            
                yield {
                    "_index": idx,
                    "document": {
                        "id": id,
                        "source": source,
                        "title": title,
                        "text": passage
                        }
                    }
            
            pbar.update()
     

In [7]:
# 36.2 = 1000
test = []

count = 0
for i in passages(idx="testing", source="wikipedia"):
    count += 1
    
    if count > 100:
            break

    test.append(i)
    
    

  0%|          | 26/17039 [00:00<10:38, 26.64it/s]


In [16]:
#len(test)
test[-10:]


[{'_index': 'testing',
  'document': {'id': '61838078',
   'source': 'wikipedia',
   'title': 'Witch &amp; Wizard (series)',
   'text': 'Synopsis. Witch &amp; Wizard. The novel follows fifteen-year old Wisty and her eighteen-year-old brother Whit in a dystopian future, where people can be brought up on charges of witchcraft. The teens and their parents are charged with witchcraft despite their claims that magic does not exist. They are taken from their parents when they begin to show magical abilities after receiving a book and drum stick.'}},
 {'_index': 'testing',
  'document': {'id': '61838078',
   'source': 'wikipedia',
   'title': 'Witch &amp; Wizard (series)',
   'text': 'The novel follows fifteen-year old Wisty and her eighteen-year-old brother Whit in a dystopian future, where people can be brought up on charges of witchcraft. The teens and their parents are charged with witchcraft despite their claims that magic does not exist. They are taken from their parents when they begin

In [17]:
from elasticsearch import Elasticsearch

# INIT OBJECTS

PORT = "http://localhost:9200"
INDEX_NAME = "knowledge"
SOURCE = "wikipedia"
errors_before_interrupt = 5
refresh_index_after_insert = False
max_insert_retries = 3
yield_ok = False

wiki_ev = Elasticsearch(
    PORT,
    #http_auth=(es_api_user, es_api_password)
    retry_on_timeout=True,  # should timeout trigger a retry on different node?
)

wiki_ev.elastic_index = INDEX_NAME

wiki_ev

<Elasticsearch(['http://localhost:9200'])>

In [20]:
# DELETE ALL

wiki_ev.delete_by_query(index=INDEX_NAME, query={"match_all": {}})

INFO:elastic_transport.transport:POST http://localhost:9200/knowledge/_delete_by_query [status:200 duration:1.024s]


ObjectApiResponse({'took': 1020, 'timed_out': False, 'total': 30000, 'deleted': 30000, 'batches': 30, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [19]:
import re
from elasticsearch import helpers
from elasticsearch.helpers import streaming_bulk, parallel_bulk
from tqdm.auto import tqdm
import spacy

# nlp = spacy.load("en_core_web_sm")

# def sentence_window(article, window=3, step=2): 
#     """ Generates a list of sentences of sliding size = window """
    
#     sents = list(nlp(article).sents)
    
#     if len(sents) == window:
#         yield str(sents)

#     for i in range(0, len(sents)):
#         yield(str(sents[i:i + window]))

# def load_data(docs, idx_):
#     """ Generates an evidence document to be inserted into ES Index """
#     for doc in docs:
#         idx, title, text = doc

#         for paragraph in sentence_window(text):
#             yield {
#                 "_index": idx_,
#                 "document": {
#                     "id": idx,
#                     "title": title,
#                     "text": paragraph
#                 }
#             }

### TEST ###
# test = []
# id, title, text = knowledge

# kw = sentence_window(knowledge[2])
# for i in kw:
#     print(i)

es = wiki_ev
errors_count = 0

# TODOs: Tune chunk size
chunk_size = 10000
counta = len(files)//chunk_size

# with tqdm(total=counta) as pbar:
    #for ok, result in parallel_bulk(es, passages(idx=INDEX_NAME, source=SOURCE), chunk_size=chunk_size, request_timeout=60*3):
successes = 0
for ok, result in tqdm(streaming_bulk(es, index=INDEX_NAME, actions=passages(idx=INDEX_NAME, source=SOURCE), 
                        chunk_size=chunk_size, request_timeout=60*3, max_retries=3)):
    if ok is not True:
            logging.error('Failed to import data')
            logging.error(str(result))
            errors_count += 1

            if errors_count == errors_before_interrupt:
                logging.fatal('Too many import errors, exiting with error code')
                exit(1)
    
    successes += ok

    #pbar.update()   
        

0it [00:00, ?it/s]

  0%|          | 0/17039 [00:00<?, ?it/s]

INFO:elastic_transport.transport:PUT http://localhost:9200/knowledge/_bulk [status:200 duration:1.654s]


KeyboardInterrupt: 

In [ ]:
query = "government emails privacy"

def search(query, k=5):
    results = es.search(
        index = es.elastic_index,
        body= {
            "size": k,
            "query": {
                "match": {
                    "document.text": query,
        }}})

    hits = results["hits"]["hits"]
    doc_ids = [row['_source']["document"]["id"] for row in hits]

    print(results)
    return (hits, doc_ids)


test = search(query, k=2)[0][0]["_source"]["document"]["text"]
test


In [ ]:
# import time
# from elasticsearch import helpers
# from elasticsearch.helpers import streaming_bulk, parallel_bulk
# from tqdm import tqdm

# "https://stackoverflow.com/questions/67522617/elasticsearch-bulk-insert-w-python-socket-timeout-error#:~:text=The%20connection%20to%20elasticsearch%20has,be%20handled%20as%20an%20error."

# "https://github.com/elastic/elasticsearch-py/issues/297"

# def load_data(docs, idx_):
#     for doc in docs:
#         idx, title, text = doc
#         doc_ = {"id": idx, "title": title, "text": text}

#         yield {
#             "_index": idx_,
#             "document": {
#                 "id": idx,
#                 "title": title,
#                 "text": text
#             }
#         }

# # TODOs: Utils, Duration Function as Decorator
# es = wiki_ev
# errors_count = 0

# # TODOs: Increase Chunk Size, with extended Timeout + handeling
# # TODOs: Experiment-Check with Yield OK
# for ok, result in parallel_bulk(es, load_data(wiki_data, "wiki_evidence"), chunk_size=500, request_timeout=60*3):
#     if ok is not True:
#             logging.error('Failed to import data')
#             logging.error(str(result))
#             errors_count += 1

#             if errors_count == errors_before_interrupt:
#                 logging.fatal('Too many import errors, exiting with error code')
#                 exit(1)

In [ ]:

# import spacy

# nlp = spacy.load("en_core_web_sm")

# def sentence_window(article, window=3, step=2): 
#     """ Generates a list of sentences of sliding size = window """
    
#     sents = list(nlp(article).sents)
    
#     if len(sents) == window:
#         yield sents

#     for i in range(0, len(sents)):
#         yield(sents[i:i + window])

# def load_data(docs, idx_):
#     """ Generates an evidence document to be inserted into ES Index """
#     for doc in docs:
#         idx, title, text = doc

#         for paragraph in sentence_window(text):
#             yield {
#                 "_index": idx_,
#                 "document": {
#                     "id": idx,
#                     "title": title,
#                     "text": paragraph
#                 }
#             }


In [ ]:
# import time
# from elasticsearch import helpers
# from elasticsearch.helpers import streaming_bulk
# from tqdm import tqdm
#
# "https://stackoverflow.com/questions/67522617/elasticsearch-bulk-insert-w-python-socket-timeout-error#:~:text=The%20connection%20to%20elasticsearch%20has,be%20handled%20as%20an%20error."
#
# def load_data(docs, idx_):
#     for doc in docs:
#         idx, title, text = doc
#         doc_ = {"id": idx, "title": title, "text": text}
#
#         yield {
#             "_index": idx_,
#             "document": doc
#         }
#
# # TODOs: Utils, Duration Function as Decorator
# es = wiki_ev
# #helpers.bulk(es, load_data(wiki_data, "wiki_evidence"), raise_on_error=False, chunk_size=500)
# errors_count = 0
# # TODOs: Increase Chunk Size, with extended Timeout + handelling
# for ok, result in streaming_bulk(es, load_data(wiki_data, "wiki_evidence"), chunk_size=500, request_timeout=60*3, yield_ok=yield_ok, refresh=refresh_index_after_insert):
#     if ok is not True:
#             logging.error('Failed to import data')
#             logging.error(str(result))
#             errors_count += 1
#
#             if errors_count == errors_before_interrupt:
#                 logging.fatal('Too many import errors, exiting with error code')
#                 exit(1)

In [ ]:
# for article in wiki_data:
#     _id, title, text = article
#     doc = {"id": _id, "title": title, "text": text}
#
#     wiki_ev.add_doc(doc)

In [ ]:
# https://github.com/elastic/elasticsearch-py/issues/297

In [ ]:
### DOCUMENT IMPORT: FULL-TEXTS ###
# from multiprocessing import Pool
# # from utils import get_contents
# import utils
# from tqdm import tqdm
# import json

# # TODOs: USE A GENERATOR OBJECT
# import spacy
# nlp = spacy.load("en_core_web_sm")
#
# p = Pool(8)
# files = [f for f in kw_sample]
#
# count = 0
# test = []
# with tqdm(total=len(files)) as pbar:
#     for documents in p.map(utils.get_contents_2, files):
#             for doc in documents:
#                 _id, title, text = doc
#
#             # count += 1
#             # doc_ = {"id": _id, "title": title, "text": text}
#
#             #wiki_ev.add_doc(doc)
#             test.append(doc_)
#     pbar.update()

In [ ]:
# from elasticsearch import Elasticsearch

# # INIT OBJECT

# # TODOs: Persist a Generator Object
# PORT = "http://localhost:9200"
# INDEX_NAME = "wiki_evidence"
# errors_before_interrupt = 5
# refresh_index_after_insert = False
# max_insert_retries = 3
# yield_ok = False

# wiki_ev = Elasticsearch(
#     PORT,
#     #http_auth=(es_api_user, es_api_password)
#     retry_on_timeout=True,  # should timeout trigger a retry on different node?
# )

# wiki_ev.elastic_index = INDEX_NAME

# wiki_ev

<Elasticsearch(['http://localhost:9200'])>

In [ ]:
# ### DOCUMENT IMPORT: SEGMENTED-TEXTS ###

# from multiprocessing import Pool
# # from utils import get_contents
# from tqdm import tqdm

# # p = Pool(8)
# # files = [f for f in kw_sample]
# #
# # count = 0
# # test = []
# # with tqdm(total=len(files)) as pbar:
# #     for documents in p.map(get_contents, files):
# #         for doc in documents:
# #             _id, title, text = doc
# #
# #             count += 1
# #             doc_ = {"id": _id, "title": title, "text": text}
# #
# #             #wiki_ev.add_doc(doc)
# #             test.append(doc)
# #     pbar.update()

In [ ]:
# ### QUERY DB ###
# import elastic_db
# # from elastic_db import ElasticDB
# #
# # # Params
# # PORT = "http://localhost:9200"
# # INDEX = "wiki_evidence"
# # DOC = "evidence"
# #
# # # Init Elasticsearch DB
# # wiki_ev_ = ElasticDB(elastic_port=PORT, elastic_index=INDEX, elastic_doc=DOC)
# #
# # results = wiki_ev_.search("exploitation a wider public debate indecency adult")
# # results

In [ ]:
### SQLITE LOAD ###
# from multiprocessing import Pool
# import utils
# from tqdm import tqdm
# import sqlite3

# import spacy
# import uuid

# nlp = spacy.load("en_core_web_sm")

# def paragraphs(document):
#     start = 0
#     document = nlp(document)
#     passages = []
#     for token in document:
#         if token.is_space and token.text.count("\n") > 1:
#             yield document[start:token.i]
#             start = token.i
#     yield document[start:]


# def get_contents(filename):
#     """Parse the contents of a file. Each line is a JSON encoded document."""
#     documents = []

#     with open(filename) as f:
#         for line in f:
#             doc = json.loads(line)

#             if doc["text"] == "": continue
#             if not doc: continue

#             passages = [str(i) for i in paragraphs(doc["text"])][0].split("\n")

#             for passage in passages:
#                 if len(passage) < 50:
#                     continue

#                 documents.append((str(uuid.uuid4()).replace('-',''), doc['id'], doc["title"], passage))

#     return documents

# save_path = "../data/wiki_evidence.db"
#
# p = Pool(8)
# files = [f for f in kw_sample]
#
# conn = sqlite3.connect(save_path)
# c = conn.cursor()
#
# documents = "documents"
# c.execute(f"CREATE TABLE documents (id PRIMARY KEY, id_, title, text);")
#
# count = 0
# step = 100
# batches = [files[i:i + step] for i in range(0, len(files), step)]
#
# for i, batch in enumerate(batches):
#     logger.info(f"[.... Batch #{i} .....]")
#     with tqdm(total=len(batch)) as pbar:
#         for document in tqdm(p.imap_unordered(get_contents, files)):
#             count += 1
#             for content in document:
#                 # _id, title, passage = content
#                 c.executemany("INSERT INTO documents VALUES (?,?,?,?)", (content,))
#
#         pbar.update()
#         logger.info(f"[Uploaded {count} documents]")
#
# conn.commit()
# conn.close()

In [41]:
# seq = [0, 1, 2, 3, 4, 5]
# window_size = 3
# step = 2

# # steps = 0, 2, 4 

# for i in range(0, len(seq) - window_size + 1, step):
#     print(i)
#     # print(i + window_size)
#     if i + window_size > len(seq):
#         # print(window_size)
#         window_size = i + window_size - len(seq)
    
#     print(seq[i: i + window_size])

0
[0, 1, 2]
2
[2, 3, 4]


In [22]:
for i in range(0, len(seq), 2):
    print(i)

0
2
4


In [48]:
seq = [0,1,2,3,4,5]

import more_itertools

test = more_itertools.windowed(seq,n=3, step=2)
list(more_itertools.windowed(seq,n=3, step=2))

[(0, 1, 2), (2, 3, 4), (4, 5, None)]